# Setup

The package is imported as `cr` in this chunk. Sub-modules can be called using:
- `cr.pp` (functions in the `crispr/processing` directory scripts)
- `cr.ax` (functions in the `crispr/processing` directory scripts)
- `cr.pl` (functions in the `crispr/visualization` directory scripts)
- `cr.tl` (functions in the `crispr/utils` directory scripts)

You can access functions this way directly (e.g., `cr.tl.print_counts()`) if they are "exposed," i.e., listed in the `__init__.py` script under a given sub-module (e.g., `crispr/processing/__init__.py`).

Other functions or objects you may need to import (e.g., the default perturbation layer name, using `from crispr.analysis.perturbations import layer_perturbation`); however, most or all functions are exposed.

There are also certain `Crispr` class initialization arguments defined here. No need to change anything***; these arguments are all just to identify columns in the CRISPR screening data and such.

You will specify arguments to your liking in the next chunk of code under the "Options" section.

---

\*** One exception may be `col_cell_type` to "leiden" or "majority_voting" if you want one of those as the default cell type label used in downstream processes, though you can always just specify something different using the `col_cell_type` argument in a given method.

In [1]:
%load_ext autoreload
%autoreload 2

import crispr as cr
from crispr.class_sc import Omics
import scanpy as sc
import copy
import os
import seaborn as sns
import matplotlib.pyplot as plt
import functools
import pandas as pd
import numpy as np
from examples import config

col_sample_id = "orig.ident"
kws_harmony = dict(plot_convergence=True, random_state=1618)
kws_init = dict(assay=None, assay_protein=None, col_sample_id=None,
                col_gene_symbols="gene_symbols",
                col_cell_type="predicted_labels",
                col_perturbed="perturbation",
                col_guide_rna="feature_call",
                col_num_umis="num_umis",
                col_condition="target_gene_name",
                key_control="NT", key_treatment="KD")

ryp2 is not installed. Install with pip install rpy2 to run tools with R support.

## Options

Here you can specify certain arguments to guide pre-processing of the scRNA-seq (e.g., filtering by MT counts) and Perturb-seq data (e.g., guide RNA filtering and target gene mapping/assignment).

### File Path

Here I specify the file paths for HH03-HH06. You can specify a dictionary of multiple files in order to integrate the samples into one object using Harmony (replacing \<DIR\> below with the path to the directory). 

I specify the `file_path` argument via code below in order to detect where your data are relative to the working directory of this script; however, in practice, you can specify paths directly/manually. 

```
file_path = {"HH03_normoxia_noMDP": {"directory": "<DIR>/HH03"},
             "HH04_normoxia_MDP": {"directory": "<DIR>/HH04"},
             "HH05_hypoxia_noMDP": {"directory": "<DIR>/HH05"}, 
             "HH06_hypoxia_MDP": {"directory": "<DIR>/HH06"}}
```

In [2]:
file_path = ["HH03_normoxia_noMDP", "HH04_normoxia_MDP",
             "HH05_hypoxia_noMDP", "HH06_hypoxia_MDP"]
file_path = dict([(x, {"directory": os.path.join(
    config.DIR, "crispr-screening", x.split("_")[0])}) for x in file_path])
file_path_list = [file_path[x] for x in file_path]

### scRNA-seq Pre-Processing & Clustering

You don't necessarily need to change these, but you can if you want to fiddle with the options. 

The main wrapper function for preprocessing is `cr.pp.process_data()`. This function calls additional functions in the same script (and some others in the visualization module, `crispr/visualization/preprocessing.py`).

<u> __Notes__ </u>  

- More options are available for greater customization. See the docstring in the functions in `crispr/processing/preprocessing.py` (the `cr.pp` module, if you imported the package using `import crispr as cr` as is done in the first chunk in this notebook) and `crispr/analysis/clustering.py`. 

- Certain highly-customized (i.e., infrequently-used) arguments are "catch-alls" that require you to specify a dictionary of arguments to be passed to a function in an external package (e.g., the `kws_neighbors` argument in the pre-processing method is passed to `scanpy.pp.neighbors()`). Refer to the relevant package's documentation for a list of potential arguments.

In [3]:
kws_umap = dict(min_dist=0.3)
kws_pp = dict(kws_hvg=dict(min_mean=0.0125, max_mean=10, min_disp=0.5), 
              target_sum=10000, cell_filter_pmt=[0, 15],
              cell_filter_ngene=[200, 7000], cell_filter_ncounts=[500, 60000], 
              gene_filter_ncell=[3, None], 
              kws_scale="z",  # z-scoring method like in the Cell paper
              regress_out=None,  # may want ["total_counts", "pct_counts_mt"]
              kws_umap=kws_umap)
kws_cluster = dict(method_cluster="leiden", kws_umap=dict(min_dist = "0.3"),
                   kws_neighbors=None, resolution=1.2,
                   kws_pca=dict(n_comps=20, use_highly_variable=True))

### Guide RNA Processing

These are arguments for gRNA filtering and target gene assignment fed to functions contained in `crispr/processing/guide_rna.py`. Note that columns with the original labels of guide RNA combinations exist in the object's anndata attributes (e.g., `selves[0].adata["feature_call_original"]`).

The main wrapper function is `cr.pp.process_guide_rna()`. 
- This function calls other functions in that same script, where you can see their code. 
- You can explore how they work individually by **importing them from the submodule** and providing appropriate inputs or by using the **debugger** to step into the function when it's called within `cr.Crispr` initialization.
- Some arguments used by this function (column names, keys) are provided via the object `kws_init` in this example because the `Crispr` class object will use these definitions across multiple contexts beyond gRNA processing (so they are stored in attributes, e.g., `self._columns["col_guide_rna"]`) so they are consistently accessible.
- Not all possible arguments are explicitly listed in the wrapper function's definition. A list of **additional keyword arguments** that can be provided can be found in documentation of the function `filter_by_guide_counts()`, to which the wrapper function feeds additional arguments not listed in its function definition. These additional arguments are also specified in the example below.

Many of these arguments are designed to implement the complex procedure requested in [Jira issue 737](https://mssm-ipm.atlassian.net/browse/CHOLAB-737?atlOrigin=eyJpIjoiYjNjMzFkNzkzZjBhNDU5Zjg0YjY5OTQ2MjM4NDI0NTQiLCJwIjoiaiJ9) (particularly, [this comment](https://mssm-ipm.atlassian.net/browse/CHOLAB-737?focusedCommentId=11589)) and (for now) have defaults reflecting that procedure for ease of remembering them. Most users not wanting to employ this particular procedure will therefore probably have to specify None, 0 (for minimum), 100 (for maximum percent arguments), etc. for certain arguments to avoid these steps.

<u> __Label Processing Arguments__ </u>  

- `feature_split`: This argument defines the character that separates multiple guide RNAs, (e.g., "|" in "STAT1-1|NOD2-2-2").
- `guide_split`: This argument defines the character that separates individual IDs within gRNAs targeting the same gene (e.g., "-" in "STAT1" and "STAT1-1-2").
- `key_control_patterns`: This is a list of patterns that count as control guides. For instance, if control guides are denoted by labels such as "NEGCTRL-1-2" and "CONTROL" and "CTRL-2" and such, you could specify `["CTRL", "CONTROL"]` if those specific strings are not found in any targeting gRNA labels.
- `key_control` is mainly aesthetic because it simply renames the control guides to something else (e.g., "Control") in the final targeting gene label column.

<u> __Guide Mapping and Filtering Arguments__ </u>  

The arguments below are listed roughly in order that the mapping/filtering steps in which they are used are performed.

First, we determine which control guides can be removed from the list of guides with which that cell is considered transfected.
- `remove_multi_transfected`: If True, multiply-transfected cells will be removed from the data, unless they are labeled as pseudo-single-transfected according to other argument specifications.
- `max_pct_control_drop`: If the percent of UMI counts that are control guides (combined across individual guide IDs, e.g., "NEGCTRL-1-1" and "NEGCTRL-1-2" and so on) is below this number, the control label will be removed from the list of guides for that cell (e.g, "CTRL|STAT1|NOD2" --> "STAT1|NOD2").
- `min_n_target_control_drop`: If the percent of control UMI counts is above `max_pct_control_drop`, but the total non-control (targeting) sgRNA UMI count is over 100, control is removed from the list of guides.

For cells that are still not considered pseudo-single-transfected after the control guide filtering described above:

- `min_pct_avg_n`: All guides whose UMIs make up less than this percentage of the total UMIs will be dropped from the list of guides.
- `min_pct_dominant`: For cells that after that step are still considered multiply-transfected, if a single guide makes up this percent or greater of the total UMIs, it is considered the dominant guide and the cell pseudo-single-transfected for the corresponding target gene.

For cells that are still considered multiply-transfected:
- `drop_multi_control`: If False, cells that (after the filtering described above) are still considered transfected with a control guide plus only one type of targeting guide will be considered pseudo-single-transfected for the target guide. Otherwise, if both control and targeting guides remain in the list of guides for that cell after all other gRNA filtering/processing, the cell will be dropped.

If `remove_multi_transfected` is True, then all cells that are still considered multiply-transfected will be dropped.

In [4]:
kws_grna = dict(feature_split="|", guide_split="-", 
                key_control_patterns=["CTRL"],
                remove_multi_transfected=True, drop_multi_control=False,
                remove_from_gex=False,
                max_pct_control_drop=75, min_n_target_control_drop=100,
                min_pct_avg_n=40, min_pct_dominant=80)

## Data Loading & Guide RNA Processing

Now, we can use those specifications to initialize an object that stores the data, attributes with information about the data and results from downstream analyses, and methods allowing you to conduct those analyses conveniently and consistently without having to 
- repeatedly specify information about your data across related functions within the same workflow/analysis,
- keep track of and pass outputs from previous processes across many steps, and
- change all your code if you decide to use different datasets later.

In [5]:
kws_multi = dict(kws_pp=kws_pp, kws_harmony={"plot_convergence": True, 
                                             "random_state": 1618},
                 kws_cluster=kws_cluster)  # pre-integration processing
self = cr.Crispr(file_path, **kws_init, kws_multi=kws_multi)  # create object



<<< INITIALIZING CRISPR CLASS OBJECT >>>



<<< INITIALIZING CRISPR CLASS OBJECT >>>


Unused keyword arguments: {'kws_process_guide_rna': None}.

col_gene_symbols="gene_symbols"
col_cell_type="leiden"
col_sample_id="unique.idents"
col_batch="unique.idents"
col_condition="target_gene_name"
col_num_umis=None
key_control="NT"
key_treatment="KD"


<<< INITIALIZING CRISPR CLASS OBJECT >>>


Unused keyword arguments: {'kws_process_guide_rna': None}.

col_gene_symbols="gene_symbols"
col_cell_type="leiden"
col_sample_id="unique.idents"
col_batch="unique.idents"
col_condition="target_gene_name"
col_num_umis=None
key_control="NT"
key_treatment="KD"

<<< LOADING PROTOSPACER METADATA >>>


Cell Counts: Raw

15078


Gene Counts: Raw



 AnnData object with n_obs × n_vars = 15078 × 36601
    obs: 'num_features', 'feature_call', 'num_umis'
    var: 'gene_ids', 'feature_types'
    layers: 'counts'


 AnnData object with n_obs × n_vars = 15078 × 36601
    obs: 'num_features', 'feature_call', 'num_um

### Check Whether Target Gene Expression Data is Available

In [ ]:
[np.mean([x in s.adata.var_names for x in s.adata.obs.target_gene_name.unique(
    )]) for s in selves]

### Set Genes to Plot

This code will simply use all target genes as the genes of interest to plot. Alternatively, you could specify genes manually in this variable (e.g., `genes = ["NOD2", "ATG16L1"]`) to plot those in the code chunks below.

In [ ]:
genes = functools.reduce(lambda i, j: i + j, [list(s.adata.obs[
    s._columns["col_target_genes"]].unique()) for s in self])
genes = list(set(genes).intersection(self.adata.var_names))

## Maintain Original Data

You can easily re-initialize a new object using this data object by passing it instead of the file path dictionary, e.g., `self =cr.Crispr(adata, **kws_init)` in order to have a full Crispr object without having to re-integrate the samples. 

In [ ]:
adata = copy.deepcopy(self.adata)

# Clustering

We need to have `kws_pca=False` so that the Harmony-adjusted PCA created during integration (rather than a newly-calculated one) will be used. 


In [ ]:
self.cluster(**{**kws_cluster, "kws_pca": False}, layer="log1p")

## CellTypist Annotations

In [ ]:
model =  "Immune_All_Low.pkl"  # choose a CellTypist model
figs_ct = self.annotate_clusters(model, copy=False)

# Plotting


<u>__These are just examples!__</u>  

The plotting functionality is highly flexible, feature-rich, and customizable. Its capabilities cannot easily be summarized here. 

<u>__If you want to create different plots, explore the documentation__</u> and other <u>__example notebooks__</u> and <u>__experiment__</u> with the functions/methods to see if your desired figures can be created.  

## Compare KD vs. NT

In [ ]:
fig, axs = self.plot_gex_targets(layer="log1p", figsize=(40, 40))

## By Cell Type

### UMAP 

In [ ]:
self.map()
self.map("NOD2")

### Violin Plots

In [ ]:
self.plot(kind="violin", genes=genes, layer="log1p")

### Matrix Plots

In [ ]:
for self in selves:
    self.plot(kind="matrix", genes=genes, layer="log1p")

## By Target Gene

### Dot Plot

In [ ]:
for self in selves:
    self.plot(kind="dot", genes=genes, layer="counts", 
              col_cell_type=self._columns["col_target_genes"],
              categories_order=genes + [self._keys["key_control"]])  # target

### Matrix Plot

In [ ]:
self.plot(kind="matrix", genes=genes, layer="log1p", 
          col_cell_type=self._columns["col_target_genes"],
          kws_matrix=dict(vcenter=0, cmap="magma"))  # ~ target

### Violin

We'll choose a random subset of the target genes to be able to see the violin plots more clearly.

In [ ]:
ggg = list(pd.Series(genes).sample(10))
self.plot(kind="violin", genes=ggg, layer="log1p",
          col_cell_type=self._columns["col_target_genes"],
          categories_order=ggg + [self._keys["key_control"]],
          kws_violin=dict(figsize=(20, 20). dendogram=False))

### Heatmaps

#### Standardize by Cell

In [ ]:
self.plot(kind="heat", genes=genes, layer="counts",
          col_cell_type=self._columns["col_target_genes"],
          kws_heat=dict(cmap="magma", swap_axes=True,
                        standard_scale="obs", figsize=(40, 30)))

#### Standardize by Gene

In [ ]:
self.plot(kind="heat", genes=genes, layer="counts",
          col_cell_type=self._columns["col_target_genes"],
          kws_heat=dict(cmap="magma", swap_axes=True,
                        standard_scale="var", figsize=(20, 20)))

# Analyses

## Distance Metrics

In [ ]:
for x in ["mmd", "edistance"]:
    _ = self.compute_distance(distance_type=x, method="X_pca", kws_plot=dict(
        figsize=(15, 15), robust=True))

## Mixscape

In [ ]:
tgis = list(self.rna.obs[self._columns["col_target_genes"]].sample(3))
_ = self.run_mixscape(target_gene_idents=tgis)  # plot subset of target genes
# _ = self.run_mixscape(target_gene_idents=True)  # plot all target genes

## Augur

In [ ]:
_ = self.run_augur(
    select_variance_features=True,  # filter by highly variable genes
    classifier="random_forest_classifier", n_folds=3, augur_mode="default", 
    kws_umap=kws_umap, subsample_size=5, kws_augur_predict=dict(span=0.7),
    n_threads=4)